In [1]:
import numpy as np
import pandas as pd
import copy
from openpyxl import load_workbook

1) Перебором параметров создать первичную структуру дата фрейма
   
    пар1 пар2 пар3 пар4 пар5 Вероятность без связей | вероятность со связями | вероятсность конф
0   Да   опц1 опц1 опц1 опц1
1   ..........................
2
3


2) Вытянуть информацию из таблицы связей или саму таблицу связи в некую структуру
3) бегать по именам фрейма используя их как ключи

In [2]:
alcohol=[["Так",0.1],["Ні",0.9]]
age=[["Дитина",0.25],["Підліток",0.35],["Юний",0.25],["Дорослий",0.1],["Літній",0.05]]
sports=[["Розвинений",0.3],["Не розвинений",0.7]]
damage_type=[["Колото-різане",0.3],["Ударно-дробляче",0.18],["Утоплення",0.18],["Тепловий удар",0.14], ["Укус тварин/комах",0.2]]
damage_area=[["Голова",0.08],["Нижні кінцівки",0.3],["Органи черевної порожнини",0.08],["Грудна клітина",0.08],["Верхні кінцівки",0.135],["Хребет",0.125],["Незастосовне",0.2]]
cause=[["Природній фактор",0.2],["Недбалість відвідувача",0.4],["Умисел відвідувача",0.15],["Халатність хазяїна",0.25]]

temp_alc=[]
temp_age=[]
temp_sports=[]
temp_dam_area=[]
temp_dam_type=[]
temp_cause=[]
tot_unbound_prob=[]
tot_bounds=[]
pxC=[]
normed_pxC=[]
for al in alcohol:
    for a in age:
        for s in sports:
            for t in damage_type:
                for ar in damage_area:
                    for c in cause:
                        temp_alc.append(al[0])
                        temp_age.append(a[0])
                        temp_sports.append(s[0])
                        temp_dam_type.append(t[0])
                        temp_dam_area.append(ar[0])
                        temp_cause.append(c[0])
                        tot_unbound_prob.append(al[1]*a[1]*s[1]*t[1]*ar[1]*c[1])
                        tot_bounds.append(np.float64(0))
                        pxC.append(np.float64(0))
                        normed_pxC.append(np.float64(0))
dict={
    "Алкогольне сп'яніння":temp_alc,
    "Вік постраждалого":temp_age,
    "Спортивний розвиток":temp_sports,
    "Тип ураження":temp_dam_type,
    "Місце ураження":temp_dam_area,
    "Причина":temp_cause,
    "P":tot_unbound_prob,
    "C":tot_bounds,
    "P*C":pxC,
    "Нормоване P*C":normed_pxC
}
conf_table=pd.DataFrame(dict)

In [3]:
print(conf_table.at[0,"Місце ураження"])

Голова


#### Розбити вхідну таблицю на блоки по парах параметрів

In [3]:
bounds=pd.read_excel('Book1.xlsx', index_col=0)
print(bounds)

                           Так   Ні  Дитина  Підліток  Юний  Дорослий  Літній  \
Дитина                    -1.0  1.0     NaN       NaN   NaN       NaN     NaN   
Підліток                  -0.7  0.7     NaN       NaN   NaN       NaN     NaN   
Юний                       0.7  0.2     NaN       NaN   NaN       NaN     NaN   
Дорослий                   0.7  0.4     NaN       NaN   NaN       NaN     NaN   
Літній                     0.5  0.6     NaN       NaN   NaN       NaN     NaN   
Розвинений                 0.0  0.0   -0.50       0.0   0.0       0.0    -0.7   
Не розвинений              0.0  0.0    0.80       0.0   0.0       0.0     0.0   
Колото-різане              0.1  0.0    0.00       0.0   0.2       0.2     0.0   
Ударно-дробляче            0.3  0.0   -0.50       0.2   0.2       0.0     0.1   
Утоплення                  0.6  0.0    0.25       0.5   0.0      -0.2     0.2   
Тепловий удар              0.4  0.0    0.20       0.0   0.0       0.0     0.2   
Укуси тварин/комах         0

In [4]:
for i in range(len(conf_table)):
    bound=1
    bound=bound*(1+bounds.at[conf_table.at[i,"Вік постраждалого"],conf_table.at[i,"Алкогольне сп'яніння"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Спортивний розвиток"],conf_table.at[i,"Алкогольне сп'яніння"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Тип ураження"],conf_table.at[i,"Алкогольне сп'яніння"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Місце ураження"],conf_table.at[i,"Алкогольне сп'яніння"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Причина"],conf_table.at[i,"Алкогольне сп'яніння"]])
    
    bound=bound*(1+bounds.at[conf_table.at[i,"Спортивний розвиток"],conf_table.at[i,"Вік постраждалого"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Тип ураження"],conf_table.at[i,"Вік постраждалого"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Місце ураження"],conf_table.at[i,"Вік постраждалого"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Причина"],conf_table.at[i,"Вік постраждалого"]])

    bound=bound*(1+bounds.at[conf_table.at[i,"Тип ураження"],conf_table.at[i,"Спортивний розвиток"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Місце ураження"],conf_table.at[i,"Спортивний розвиток"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Причина"],conf_table.at[i,"Спортивний розвиток"]])    

    bound=bound*(1+bounds.at[conf_table.at[i,"Місце ураження"],conf_table.at[i,"Тип ураження"]])
    bound=bound*(1+bounds.at[conf_table.at[i,"Причина"],conf_table.at[i,"Тип ураження"]])

    bound=bound*(1+bounds.at[conf_table.at[i,"Причина"],conf_table.at[i,"Місце ураження"]])

    conf_table.at[i,"C"]=bound

KeyError: 'Укус тварин/комах'

In [8]:
k=450
print(conf_table.iloc[450])


Алкогольне сп'яніння                   Так
Вік постраждалого                 Підліток
Спортивний розвиток          Не розвинений
Тип ураження               Ударно-дробляче
Місце ураження                      Голова
Причина                 Умисел відвідувача
P                                 0.000053
C                                  2.50614
P*C                               0.000133
Нормоване P*C                     0.000057
Name: 450, dtype: object


In [6]:
for i in range(len(conf_table)):
    conf_table.at[i,"P*C"]=conf_table.at[i,"P"]*conf_table.at[i,"C"]

In [7]:
norm=0
for i in range(len(conf_table)):
    norm=norm+conf_table.at[i,"P*C"]
for i in range(len(conf_table)):
    conf_table.at[i,"Нормоване P*C"]=conf_table.at[i,"P*C"]/norm

In [8]:
conf_table.to_excel("output.xlsx", index=False)

In [9]:

temp_df=pd.read_excel(
    "Форін МСС 2.xlsx",
    sheet_name="Sheet1",
    usecols="A:B",
    skiprows=1,
    nrows=2,
    header=None
)
for i in range(len(temp_df)):
    temp_df.at[i,1]=0
alc_ser=copy.deepcopy(pd.Series(data=temp_df.iloc[:,1].values,index=temp_df.iloc[:,0].values))

temp_df=pd.read_excel(
    "Форін МСС 2.xlsx",
    sheet_name="Sheet1",
    usecols="C:D",
    skiprows=1,
    nrows=5,
    header=None
)
for i in range(len(temp_df)):
    temp_df.at[i,3]=0
age_ser=copy.deepcopy(pd.Series(data=temp_df.iloc[:,1].values,index=temp_df.iloc[:,0].values))
temp_df=pd.read_excel(
    "Форін МСС 2.xlsx",
    sheet_name="Sheet1",
    usecols="E:F",
    skiprows=1,
    nrows=2,
    header=None
)
for i in range(len(temp_df)):
    temp_df.at[i,5]=0
sport_ser=copy.deepcopy(pd.Series(data=temp_df.iloc[:,1].values,index=temp_df.iloc[:,0].values))
temp_df=pd.read_excel(
    "Форін МСС 2.xlsx",
    sheet_name="Sheet1",
    usecols="G:H",
    skiprows=1,
    nrows=5,
    header=None
)
for i in range(len(temp_df)):
    temp_df.at[i,7]=0
type_ser=copy.deepcopy(pd.Series(data=temp_df.iloc[:,1].values,index=temp_df.iloc[:,0].values))
temp_df=pd.read_excel(
    "Форін МСС 2.xlsx",
    sheet_name="Sheet1",
    usecols="I:J",
    skiprows=1,
    nrows=7,
    header=None
)
for i in range(len(temp_df)):
    temp_df.at[i,9]=0
area_ser=copy.deepcopy(pd.Series(data=temp_df.iloc[:,1].values,index=temp_df.iloc[:,0].values))
temp_df=pd.read_excel(
    "Форін МСС 2.xlsx",
    sheet_name="Sheet1",
    usecols="K:L",
    skiprows=1,
    nrows=4,
    header=None
)
for i in range(len(temp_df)):
    temp_df.at[i,11]=0
cause_ser=copy.deepcopy(pd.Series(data=temp_df.iloc[:,1].values,index=temp_df.iloc[:,0].values))

In [10]:
print(type_ser)

Колото-різане         0.0
Ударно-дробляче       0.0
Утоплення             0.0
Тепловий удар         0.0
Укуси тварин/комах    0.0
dtype: float64


In [11]:
for i in range(len(conf_table)):
    for key in alc_ser.index:
        if conf_table.at[i,"Алкогольне сп'яніння"]==key:
            alc_ser[key]=alc_ser[key]+conf_table.at[i,"Нормоване P*C"]
    for key in age_ser.index:
        if conf_table.at[i,"Вік постраждалого"]==key:
            age_ser[key]=age_ser[key]+conf_table.at[i,"Нормоване P*C"]
    for key in sport_ser.index:
        if conf_table.at[i,"Спортивний розвиток"]==key:
            sport_ser[key]=sport_ser[key]+conf_table.at[i,"Нормоване P*C"]
    for key in type_ser.index:
        if conf_table.at[i,"Тип ураження"]==key:
            type_ser[key]=type_ser[key]+conf_table.at[i,"Нормоване P*C"]
    for key in area_ser.index:
        if conf_table.at[i,"Місце ураження"]==key:
            area_ser[key]=area_ser[key]+conf_table.at[i,"Нормоване P*C"]
    for key in cause_ser.index:
        if conf_table.at[i,"Причина"]==key:
            cause_ser[key]=cause_ser[key]+conf_table.at[i,"Нормоване P*C"]
    # прописать также для остальных серий 

In [12]:
ser_listed=[alc_ser,age_ser,sport_ser,type_ser,area_ser,cause_ser]

In [13]:
wb=load_workbook("output.xlsx")
ws=wb["Sheet2"]

start_row=2

for start_col in range(len(ser_listed)):
    for i, (index, value) in enumerate(ser_listed[start_col].items()):
        ws.cell(row=start_row+i, column=2*start_col+1, value=index)
        ws.cell(row=start_row+i, column=2*start_col+2,value=value)

wb.save("output.xlsx")

In [14]:
dict1={
    "Регулярне прибирання пляжу":np.ones(len(conf_table)),
    "Заборона побудови вишок для стрибків":np.ones(len(conf_table)),
    "Моніторинг популяції отруйних тварин":np.ones(len(conf_table)),
    "Інструктажі з техніки безпеки у ЗО":np.ones(len(conf_table)),
    "Заборона алкоголю":np.ones(len(conf_table)),
    "Створення місць для охолодження":np.ones(len(conf_table))
}
dict2={
    "Алкогольне сп'яніння":temp_alc,
    "Вік постраждалого":temp_age,
    "Спортивний розвиток":temp_sports,
    "Тип ураження":temp_dam_type,
    "Місце ураження":temp_dam_area,
    "Причина":temp_cause,
}
configurs=pd.DataFrame(dict2)
conf_table_2R=pd.DataFrame(dict1)
print(configurs)

     Алкогольне сп'яніння Вік постраждалого Спортивний розвиток  \
0                     Так            Дитина          Розвинений   
1                     Так            Дитина          Розвинений   
2                     Так            Дитина          Розвинений   
3                     Так            Дитина          Розвинений   
4                     Так            Дитина          Розвинений   
...                   ...               ...                 ...   
2795                   Ні            Літній       Не розвинений   
2796                   Ні            Літній       Не розвинений   
2797                   Ні            Літній       Не розвинений   
2798                   Ні            Літній       Не розвинений   
2799                   Ні            Літній       Не розвинений   

           Тип ураження  Місце ураження                 Причина  
0         Колото-різане          Голова        Природній фактор  
1         Колото-різане          Голова  Недбалість відвідувача

In [15]:
print(conf_table_2R)

      Регулярне прибирання пляжу  Заборона побудови вишок для стрибків  \
0                            1.0                                   1.0   
1                            1.0                                   1.0   
2                            1.0                                   1.0   
3                            1.0                                   1.0   
4                            1.0                                   1.0   
...                          ...                                   ...   
2795                         1.0                                   1.0   
2796                         1.0                                   1.0   
2797                         1.0                                   1.0   
2798                         1.0                                   1.0   
2799                         1.0                                   1.0   

      Моніторинг популяції отруйних тварин  \
0                                      1.0   
1                  

In [16]:
bounds_2=pd.read_excel("Book1.xlsx", sheet_name="Sheet2", index_col=0)
print(bounds_2)

                           Регулярне прибирання пляжу  \
Так                                               0.0   
Ні                                                0.0   
Дитина                                            0.2   
Підліток                                          0.0   
Юний                                              0.0   
Дорослий                                          0.0   
Літній                                            0.0   
Розвинений                                        0.0   
Не розвинений                                     0.0   
Колото-різане                                     0.9   
Ударно-дробляче                                  -0.5   
Утоплення                                        -0.5   
Тепловий удар                                    -1.0   
Укус тварин/комах                                -1.0   
Голова                                           -0.5   
Нижні кінцівки                                    0.9   
Органи черевної порожнини      

In [17]:
for i in range(len(conf_table_2R)):
    for col in conf_table_2R.columns:
        for conf in configurs.columns:
            conf_table_2R.at[i,col]=conf_table_2R.at[i,col]*(1+bounds_2.at[configurs.at[i,conf],col])

In [18]:
with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    conf_table_2R.to_excel(writer, sheet_name="Sheet3", index=False)

In [19]:
sums_to_norm=[]
for i, row in conf_table_2R.iterrows():
    sums_to_norm.append(row.sum())
print(sums_to_norm)
sums_to_norm=np.array(sums_to_norm)
print(sums_to_norm)

[np.float64(12.14397), np.float64(15.078419999999998), np.float64(7.553069999999999), np.float64(15.265349999999998), np.float64(14.92717), np.float64(14.070419999999999), np.float64(6.713069999999999), np.float64(20.753349999999998), np.float64(10.914369999999998), np.float64(14.070419999999999), np.float64(6.713069999999999), np.float64(13.45735), np.float64(10.914369999999998), np.float64(14.070419999999999), np.float64(6.713069999999999), np.float64(13.45735), np.float64(15.153569999999998), np.float64(15.078419999999998), np.float64(7.553069999999999), np.float64(20.73735), np.float64(11.64237), np.float64(15.078419999999998), np.float64(7.553069999999999), np.float64(14.353349999999999), np.float64(11.61797), np.float64(16.08642), np.float64(8.39307), np.float64(13.88135), np.float64(8.86665), np.float64(20.0766), np.float64(12.02175), np.float64(20.046750000000003), np.float64(9.114650000000001), np.float64(19.71858), np.float64(11.60679), np.float64(21.50315), np.float64(8.0586

In [20]:
for i in range(len(conf_table_2R)):
    for col in conf_table_2R.columns:
        conf_table_2R.at[i,col]=conf_table_2R.at[i,col]/sums_to_norm[i]

In [21]:
with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    conf_table_2R.to_excel(writer, sheet_name="Sheet3", index=False, startrow=0, startcol=6)

In [22]:
for i in range(len(conf_table)):
    for col in conf_table_2R.columns:
        conf_table_2R.at[i,col]=conf_table_2R.at[i,col]*conf_table.at[i,"Нормоване P*C"]

In [23]:
with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    conf_table_2R.to_excel(writer, sheet_name="Sheet3", index=False, startrow=0, startcol=12)

In [24]:
temp_sol=[]
temp_success=[]
for col in conf_table_2R.columns:
    temp_sol.append(col)
    temp_success.append(conf_table_2R[col].sum())
dict3={
    "Рішення":temp_sol,
    "Успіх": temp_success
}
result_df=pd.DataFrame(dict3)
with pd.ExcelWriter("output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    result_df.to_excel(writer, sheet_name="Sheet4", index=False)

In [25]:
print(result_df["Успіх"].sum())

0.9999999999999994
